In [1]:
import torch
import torchvision
from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.backbone_utils import mobilenet_backbone

from torchvision.models.detection.anchor_utils import AnchorGenerator
nn = torch.nn
import os
import numpy as np
from src.utils.audio_utils import compute_mel_spectrogram, compute_spectrogram, load_audio_file
from src.utils import path_utils


/Users/paulbp_dty/Desktop/P3_BBF/audio_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:

def rcnn_pretrained_backbone(num_classes : int, anchor_sizes : tuple, aspect_ratios : tuple, parameters : dict = {}):
    """
    Return a faster rcnn with a mobilenetv3 backbone.

    Args:
        num_classes (int): Number of classes expected to return (background should be taken into account)
        parameters (dict, optional): Dictionnary for the different following parameters. Defaults to {}.

    Returns:
        (torchvision.models.detection.faster_rcnn.FasterRCNN): Model implementation of pytorch

    >>> training
    # RPN
    rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
    rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
    rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
        considered as positive during training of the RPN.
    rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
        considered as negative during training of the RPN.
    rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
        for computing the loss
    rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
        of the RPN

    # Head
    box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
        considered as positive during training of the classification head
    box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
        considered as negative during training of the classification head
    box_batch_size_per_image (int): number of proposals that are sampled during training of the
        classification head
    box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
        of the classification head



    >>> inference
    # RPN
    rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
    rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
    rpn_score_thresh (float): during inference, only return proposals with a classification score
        greater than rpn_score_thresh
    rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals

    # Head
    box_score_thresh (float): during inference, only return proposals with a classification score
        greater than box_score_thresh

    box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
    box_detections_per_img (int): maximum number of detections per image, for all classes.

    """

    pretrained_backbone=True
    trainable_backbone_layers=6 # All backbone is trainable

    backbone = mobilenet_backbone("mobilenet_v3_large", pretrained_backbone, True, trainable_layers=trainable_backbone_layers)

    model = torchvision.models.detection.faster_rcnn.FasterRCNN(backbone,
                                                                num_classes,
                                                                rpn_anchor_generator=AnchorGenerator(anchor_sizes, aspect_ratios),
                                                                **parameters)

    # Custum transform ie no transform (only postprocessing)
    model.transform = GeneralizedRCNNTransform(min_size=489, max_size=2000, image_mean=[0, 0, 0], image_std=[1, 1, 1])
    return model

In [4]:
def rcnn_pretrained_backbone_train(num_classes : int, anchor_sizes : tuple, aspect_ratios : tuple, parameters : dict = {}):
    """
    Return a jit compiled faster rcnn with a mobilenetv3 backbone.
    """
    model = rcnn_pretrained_backbone(num_classes, anchor_sizes, aspect_ratios, parameters)
    model._has_warned = True # Remove warning about "RCNN always returns a (Losses, Detections) tuple in scripting"
    return torch.jit.script(model)

In [5]:
def loss_faster_rcnn(dict_losses : dict, training_rpn : bool, training_head : bool):
    """ Reduce the dictionnary of losses

    Args:
        dict_losses (dict): Dictionnary of losses
        training_rpn (bool): Bool to train the rpn
        training_head (bool): Bool to train the head

    Returns:
        (torch.Tensor): Loss
    """
    # Dict("loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg")
    loss = torch.zeros((), dtype=torch.float32, device=device)
    if training_rpn:
        loss += dict_losses["loss_objectness"]
        loss += dict_losses["loss_rpn_box_reg"]
    if training_head:
        loss += dict_losses["loss_classifier"]
        loss += dict_losses["loss_box_reg"]

In [6]:
def average_losses(dataloader, mean_loss):
    mean_loss["loss_objectness"] /= len(dataloader)
    mean_loss["loss_rpn_box_reg"] /= len(dataloader)
    mean_loss["loss_classifier"] /= len(dataloader)
    mean_loss["loss_box_reg"] /= len(dataloader)
    return mean_loss

def accumulate_losses(mean_loss, losses):
    mean_loss["loss_objectness"] += losses["loss_objectness"].item()
    mean_loss["loss_rpn_box_reg"] += losses["loss_rpn_box_reg"].item()
    mean_loss["loss_classifier"] += losses["loss_classifier"].item()
    mean_loss["loss_box_reg"] += losses["loss_box_reg"].item()

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler, macro_batch=1):
    # Initialize training
    model.train()
    optimizer.zero_grad()

    mean_loss = {"loss_objectness": 0., "loss_rpn_box_reg": 0., "loss_classifier": 0., "loss_box_reg": 0.}

    # Iterate over the dataset
    for batch, (X, targets) in enumerate(dataloader):
        # Work with the GPU if available
        X = list(x.to(device) for x in X)
        targets = list({k: v.to(device) for k, v in t.items()} for t in targets)

        # Compute prediction error
        losses, detections = model(X, targets)
        accumulate_losses(mean_loss, losses)
        loss = loss_fn(losses)

        # Backpropagation
        loss.backward()
        if (batch+1) % macro_batch == 0 or batch == len(dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

        # Print metrics
        if batch % 30 == 0 or batch == len(dataloader) - 1:
            loss_value, current = loss.item(), (batch+1) * len(X)
        print(loss_value)
    return average_losses(dataloader, mean_loss)

In [8]:
class OneShotDataset(torch.utils.data.Dataset):
    def __init__(self, detection_dir,transform_audio, transform_image, device):
        self.detection_dir=detection_dir
        self.device=device
        self.samplepath=os.path.join(self.detection_dir, 'samples')
        self.targetpath=os.path.join(self.detection_dir, 'target')
        self.indexes=os.listdir(self.samplepath).split('.')[0]
        self.transform_audio = transform_audio
        self.transform_image = transform_image
    def __len__(self):
        return len(os.listdir((self.samplepath)))
    def __get_item__(self,idx):
        index=self.indexes(idx)
        data,fs= load_audio_file(os.path.join(self.samplepath,index,'.wav'))
        target=np.load(os.path.join(self.targetpath,index,'.npy'))
        x = self.transform_audio(data)
        x = self.transform_image(x)
        return x, target

In [10]:
def transform_audio(data):
    _, _, specto = compute_spectrogram(data, 24000, nperseg=256, noverlap=256/2, scale="dB")
    # freq clip
    specto = specto[:120, :]
    return specto
    
detection_dataloader = torch.utils.data.DataLoader(OneShotDataset(detection_dir=path_utils.get_detection_data_path(),
                                                    transform_audio=transform_audio, transform_image=transform_image, device=device),batch_size=1,drop_last=True)
                                                    



TypeError: OneShotDataset.__init__() got an unexpected keyword argument 'window_size'